<a href="https://colab.research.google.com/github/saladpalad/bigML-assessment/blob/main/bigML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
!pip install spuco

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms
import tqdm as tqdm
import spuco.datasets

### Initialize dataset

In [ ]:
from spuco.datasets import SpuCoMNIST, SpuriousFeatureDifficulty
import torchvision.transforms as T

classes = [[0, 1], [2, 3], [4, 5], [6, 7], [8, 9]]
difficulty = SpuriousFeatureDifficulty.MAGNITUDE_LARGE

trainset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    spurious_correlation_strength=0.995,
    classes=classes,
    split="train"
)
trainset.initialize()

testset = SpuCoMNIST(
    root="/data/mnist/",
    spurious_feature_difficulty=difficulty,
    classes=classes,
    split="test"
)
testset.initialize()

100%|██████████| 10000/10000 [00:01<00:00, 6779.01it/s]


### Train ERM Model (ResNet-18)

In [ ]:
from spuco.models import model_factory
from spuco.utils import Trainer
from torch.optim import SGD

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model_factory("resnet18", trainset[0][0].shape, trainset.num_classes).to(device)
erm = Trainer(
    trainset=trainset,
    model=model,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)

erm.train(1)

cuda


Epoch 0: 100%|██████████| 751/751 [00:23<00:00, 32.22batch/s, accuracy=75.0%, loss=0.371]


In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:13,  1.75it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:01<00:11,  1.96it/s]

Group (0, 1) Accuracy: 31.914893617021278


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:10,  2.08it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:11<00:00,  2.08it/s]

Group (4, 3) Accuracy: 6.0606060606060606


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:12<00:00,  2.01it/s]

Group (4, 4) Accuracy: 100.0


{(0, 0): 100.0,
 (0, 1): 31.914893617021278,
 (0, 2): 37.35224586288416,
 (0, 3): 30.73286052009456,
 (0, 4): 27.659574468085108,
 (1, 0): 20.78239608801956,
 (1, 1): 100.0,
 (1, 2): 0.0,
 (1, 3): 0.0,
 (1, 4): 0.9803921568627451,
 (2, 0): 24.266666666666666,
 (2, 1): 0.0,
 (2, 2): 100.0,
 (2, 3): 0.26666666666666666,
 (2, 4): 0.0,
 (3, 0): 8.793969849246231,
 (3, 1): 1.0075566750629723,
 (3, 2): 9.06801007556675,
 (3, 3): 100.0,
 (3, 4): 11.335012594458439,
 (4, 0): 58.69017632241814,
 (4, 1): 0.2518891687657431,
 (4, 2): 0.0,
 (4, 3): 6.0606060606060606,
 (4, 4): 100.0}

### Cluster in the feature space of the ERM trained model

In [ ]:
from spuco.group_inference import Cluster, ClusterAlg

logits = erm.get_trainset_outputs()
cluster = Cluster(
    Z=logits,
    class_labels=trainset.labels,
    cluster_alg=ClusterAlg.KMEANS,
    num_clusters=2,
    device=device,
    verbose=True
)
group_partition = cluster.infer_groups()

Clustering class-wise: 100%|██████████| 5/5 [00:00<00:00, 10.49it/s]


### Retrain using "Group-Balancing"

In [ ]:
from torch.optim import SGD
from spuco.robust_train import GroupBalanceBatchERM, ClassBalanceBatchERM
from spuco.models import model_factory

model = model_factory("resnet18", trainset[0][0].shape, trainset.num_classes).to(device)
group_balance_erm = GroupBalanceBatchERM(
    model=model,
    num_epochs=5,
    trainset=trainset,
    group_partition=group_partition,
    batch_size=64,
    optimizer=SGD(model.parameters(), lr=1e-3, weight_decay=5e-4, momentum=0.9, nesterov=True),
    device=device,
    verbose=True
)
group_balance_erm.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Epoch 4: 100%|██████████| 751/751 [00:24<00:00, 30.72batch/s, accuracy=100.0%, loss=0.126]


In [ ]:
from spuco.evaluate import Evaluator

evaluator = Evaluator(
    testset=testset,
    group_partition=testset.group_partition,
    group_weights=trainset.group_weights,
    batch_size=64,
    model=model,
    device=device,
    verbose=True
)
evaluator.evaluate()

print("Worst group accuracy:", evaluator.worst_group_accuracy)
print("Avg group accuracy:", evaluator.average_accuracy)
print("Spurious attribute accuracy", evaluator.evaluate_spurious_attribute_prediction())

Evaluating group-wise accuracy:   4%|▍         | 1/25 [00:00<00:10,  2.21it/s]

Group (0, 0) Accuracy: 100.0


Evaluating group-wise accuracy:   8%|▊         | 2/25 [00:00<00:10,  2.20it/s]

Group (0, 1) Accuracy: 55.319148936170215


Evaluating group-wise accuracy:  12%|█▏        | 3/25 [00:01<00:09,  2.22it/s]

...

Evaluating group-wise accuracy:  96%|█████████▌| 24/25 [00:11<00:00,  2.16it/s]

Group (4, 3) Accuracy: 49.24242424242424


Evaluating group-wise accuracy: 100%|██████████| 25/25 [00:12<00:00,  2.03it/s]

Group (4, 4) Accuracy: 100.0
Worst group accuracy: ((2, 4), 6.149732620320855)
Avg group accuracy: 99.70374144981233


Spurious attribute accuracy 63.83
